<font size="6"><code style="background:teal;color:white">Проект 2: Воронка продаж</code></font>

Анализируем приложение, которое представляет из себя сборник фитнес-упражнений.
В нем есть несколько основных событий — некие этапы, через которые проходит пользователь приложения (по условию обучение - не обязательный этап):  
- Регистрация (**Registration**)  
- Старт обучения (**Tutorial_start**)  
- Завершение обучения (**Tutorial_finish**)  
- Выбор уровня сложности тренировок (**Level_choice**)  
- Выбор тренировок (**Training_choice**)    
- Покупка платных тренировок (**Purchase**)  

Данные находятся в двух таблицах **Event** (данные о событиях) и **Purchase** (данные о покупках) на сервере обучающей платформы Skillfactory (необходимо подключение к БД).  

Анализируемый временной интервал: **1.01.2018 - 31.12.2018**  
<div class="alert alert-block alert-warning"><b>Задачи и гипотезы:</b><ol><li>Отличается ли время прохождения различных этапов для пользователей, которые прошли обучение, от пользователей, не начинавших обучение? Насколько обучение сокращает время прохождения этапов?</li><br><li>Существует ли зависимость между вероятностью оплаты тренировки и количеством обучений, которые начинал или завершал пользователь. Нужно доказать, что успешное обучение влияет на оплату само по себе, без разницы, какое оно было по порядку.</li><br><li>Как часто пользователи начинают обучение после того, как они выбрали уровень сложности тренировок? Это позволит нам понять, насколько процесс работы с приложением понятен для пользователей: если пользователи после выбора уровня сложности обращаются к обучению, значит, работа с приложением непонятна.</li></ol></div>

**<mark>ЭТАП 1</mark>**Загружаем необходимые для работы библиотеки, сами данные и подготавливаем их для последующего анализа.

In [1]:
#Импортируем нужные библиотеки
import pandas as pd
import psycopg2
import psycopg2.extras 
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objects as go
init_notebook_mode(connected=True)
from plotly.subplots import make_subplots

In [2]:
#Пишем функцию getEventsData() - получить данные по событиям пользователей, которы зарегистрировались в 2018 году
def getEventsData():
    query = '''SELECT e.* FROM case8.events e
    WHERE e.user_id in
    (SELECT DISTINCT(user_id) 
    FROM case8.events 
    WHERE event_type = 'registration' 
    AND start_time >= '2018-01-01'
    AND start_time < '2019-01-01')
    '''.format()
    conn = psycopg2.connect(dbname='skillfactory', 
                            user='skillfactory', 
                            host='84.201.134.129',
                            password='cCkxxLVrDE8EbvjueeMedPKt',
                            port=5432)
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    dict_cur.close()
    conn.close()
    return data

In [3]:
#Пишем функцию getPurchaseData() - получить данные по оплатам пользователей, которы зарегистрировались в 2018 году
def getPurchaseData():
    query = '''SELECT p.* FROM case8.purchase p
    WHERE p.user_id in
    (SELECT DISTINCT(user_id) 
    FROM case8.events 
    WHERE event_type = 'registration' 
    AND start_time >= '2018-01-01'
    AND start_time < '2019-01-01')
    '''.format()
    conn = psycopg2.connect(dbname='skillfactory',
                            user='skillfactory',
                            host='84.201.134.129',
                            password='cCkxxLVrDE8EbvjueeMedPKt',
                            port=5432)
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    dict_cur.close()
    conn.close()
    return data

In [4]:
#Записываем полученные в результате запроса данные в соответствующие датафреймы
events_df = pd.DataFrame(getEventsData())
purchase_df = pd.DataFrame(getPurchaseData())

In [5]:
#Посмотрим на начало нашего датафрейм events_df
events_df.head()

,event_type,selected_level,start_time,tutorial_id,user_id,id
0,registration,None,2018-01-01 04:51:58,NaN,47758,147264
1,registration,None,2018-01-01 08:32:05,NaN,47759,147268
2,registration,None,2018-01-01 09:30:10,NaN,47760,147269
3,registration,None,2018-01-01 09:39:27,NaN,47761,147270
4,registration,None,2018-01-01 11:41:27,NaN,47762,147271


In [6]:
#Посмотрим на информацию об этом датафрейме
events_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82779 entries, 0 to 82778
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   event_type      82779 non-null  object        
 1   selected_level  10198 non-null  object        
 2   start_time      82779 non-null  datetime64[ns]
 3   tutorial_id     41201 non-null  float64       
 4   user_id         82779 non-null  int64         
 5   id              82779 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 3.8+ MB


Как видим наши данные содержат 82 779 строк, 6 столбцов, есть пропуски в столбцах: selected_level, tutorial_id

In [7]:
#Проверим наш датафрейм на наличие дубликатов:
events_df.drop_duplicates().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82779 entries, 0 to 82778
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   event_type      82779 non-null  object        
 1   selected_level  10198 non-null  object        
 2   start_time      82779 non-null  datetime64[ns]
 3   tutorial_id     41201 non-null  float64       
 4   user_id         82779 non-null  int64         
 5   id              82779 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 4.4+ MB


In [8]:
#Исследуем пропуски, посмотрим на event_df, где event_type=level_choice
events_df[events_df['event_type'] == 'level_choice'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10198 entries, 13 to 82775
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   event_type      10198 non-null  object        
 1   selected_level  10198 non-null  object        
 2   start_time      10198 non-null  datetime64[ns]
 3   tutorial_id     0 non-null      float64       
 4   user_id         10198 non-null  int64         
 5   id              10198 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 557.7+ KB


Как видно, этот срез датафрейма не содержит пропущенных значений в столбце selected_level, но зато содержит пропуски в tutorial_id. Это связано с тем, что для событий типа level_choice не предусмотрена запись параметра tutorial_id.

In [9]:
#Проверим аналогичные данные при условии, что срез будет содержать данные о событиях tutorial_start и tutorial_finish:
events_df[events_df['event_type'].isin(['tutorial_start','tutorial_finish'])].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41201 entries, 6 to 82778
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   event_type      41201 non-null  object        
 1   selected_level  0 non-null      object        
 2   start_time      41201 non-null  datetime64[ns]
 3   tutorial_id     41201 non-null  float64       
 4   user_id         41201 non-null  int64         
 5   id              41201 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 2.2+ MB


Столбец selected_level в таком срезе не содержит ни одного значения. Это связано с тем, что для событий, связанных с обучением, отсутствуют параметры выбора уровня сложности selected_level. А вот остальные столбцы заполнены полностью.

In [10]:
#Исследуем значения методом describe(), сначала числовые значения
events_df.describe()

,tutorial_id,user_id,id
count,41201.000000,82779.000000,82779.000000
mean,60853.276207,59932.814446,188656.000568
std,6525.340246,7074.775488,23896.384413
min,49555.000000,47758.000000,147264.000000
25%,55199.000000,53769.500000,167961.500000
50%,60863.000000,59943.000000,188656.000000
75%,66510.000000,66061.000000,209350.500000
max,72153.000000,72217.000000,230056.000000


Поскольку это всего лишь идентификационные номера - эта таблица для нас малоинформативна

In [11]:
#Исследуем объекты и дату
events_df.describe(include=['object', 'datetime64[ns]'])

,event_type,selected_level,start_time
count,82779,10198,82779
unique,5,3,82622
top,registration,medium,2018-03-01 09:53:30
freq,24460,5776,4
first,NaN,NaN,2018-01-01 04:51:58
last,NaN,NaN,2019-01-01 09:03:24


Из данной информации мы можем видеть, что существуют 5 различных вариантов событий (чаще всего событие registration), 3 уровня выбора сложности тренировок (чаще всего выбирают уровень medium). Также мы видим, что во временной интервал попадают события, на первый взгляд, выходящие за анализируемый период. Однако, на самом деле, временной интервал брался для момента регистрации и последующие события могли попасть в начало следующего года. Убедимся, что регистрации происходили в нужное нам время:

In [12]:
print(events_df[events_df['event_type']=='registration'].start_time.min())
print(events_df[events_df['event_type']=='registration'].start_time.max())

2018-01-01 04:51:58
2018-12-31 23:58:13


In [13]:
#Исследуем уникальные события в колонках event_type и selected_level:
events_df['event_type'].unique()

array(['registration', 'tutorial_start', 'tutorial_finish',
       'level_choice', 'training_choice'], dtype=object)

In [14]:
#Иссследуем возможные уровни тренировок:
events_df['selected_level'].unique()

array([None, 'hard', 'easy', 'medium'], dtype=object)

In [15]:
#Определим число уникальных пользователей:
events_df['user_id'].nunique()

24460

In [16]:
#Посмотрим на датафрейм с оплатами
purchase_df.head()

,user_id,start_time,amount,id
0,67986,2018-11-17 13:18:35,50,20103
1,58528,2018-06-23 23:24:43,100,19351
2,61351,2018-08-08 09:40:36,100,19565
3,48037,2018-01-10 13:08:34,50,18470
4,67987,2018-11-13 22:51:38,100,20087


In [17]:
#Посмотрим на информацию об этом датафрейме
purchase_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1974 entries, 0 to 1973
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     1974 non-null   int64         
 1   start_time  1974 non-null   datetime64[ns]
 2   amount      1974 non-null   int64         
 3   id          1974 non-null   int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 61.8 KB


Как видим наши данные содержат 1974 строки и 4 столбца, пропуски отсутствуют.

In [18]:
#Проверим наш датафрейм на наличие дубликатов:
purchase_df.drop_duplicates().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1974 entries, 0 to 1973
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     1974 non-null   int64         
 1   start_time  1974 non-null   datetime64[ns]
 2   amount      1974 non-null   int64         
 3   id          1974 non-null   int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 77.1 KB


In [19]:
#Исследуем численные данные:
purchase_df.describe()

,user_id,amount,id
count,1974.000000,1974.000000,1974.000000
mean,59962.116515,107.801418,19432.507092
std,7033.436103,56.747492,570.020623
min,47771.000000,25.000000,18444.000000
25%,53891.500000,50.000000,18939.250000
50%,59930.000000,100.000000,19432.500000
75%,65953.750000,150.000000,19925.750000
max,72212.000000,350.000000,20427.000000


Из описания для нас более всего информативен столбец с суммами оплаты - средний чек составил 107,8 рублей (точно единица измерения в описании проекта не дана, предполагаем исходя из решения предыдущих задач), платежи варьировались от 25 до 350 рублей. Всего было совершено 1974 транзакции принесшие выручку:

In [20]:
purchase_df.amount.sum()

212800

In [21]:
#Посмотрим сколько пользователей совершали покупки:
purchase_df.user_id.nunique()

1974

In [22]:
#В процентном отношении от всех пользователей покупателей:
print(round((purchase_df.user_id.nunique()/events_df['user_id'].nunique()*100), 2), '%')

8.07 %


Интересно, что в 8 кейсе, где анализировались данные за 2017 год, процент пользователей оплативших тренировки составлял 8,03% - показатель практически остался неизменным.

In [23]:
#Исследуем численные данные:
purchase_df.describe(include='datetime64[ns]')

,start_time
count,1974
unique,1973
top,2018-04-19 15:42:47
freq,2
first,2018-01-03 07:30:09
last,2019-01-08 00:58:57


Видим, что первая покупка была совершена 3 января 2018 и последняя 8 января 2019 года (при этом извлекали данные, чтобы регистрация происходила в интересующий нас период). В данном обзоре также видим, что по частоте попало событие 19 апреля 2018 года, однако значение частоты в два события просто повествует, что получилось точное совпадение по транзакции вплоть до минут и секунд в двух фактах оплат.

In [24]:
#Также для дальнейшей работы нам понадобятся два объединенных датафрейма:
purchase_df['event_type'] = 'purchase'
events_df = events_df.rename(columns={'id':'event_id'})
purchase_df = purchase_df.rename(columns={'id':'purchase_id'})
total_events_df = pd.concat([events_df,purchase_df],sort=False).sort_values('start_time')
total_events_df.head()

,event_type,selected_level,start_time,tutorial_id,user_id,event_id,amount,purchase_id
0,registration,None,2018-01-01 04:51:58,NaN,47758,147264.0,NaN,NaN
1,registration,None,2018-01-01 08:32:05,NaN,47759,147268.0,NaN,NaN
2,registration,None,2018-01-01 09:30:10,NaN,47760,147269.0,NaN,NaN
3,registration,None,2018-01-01 09:39:27,NaN,47761,147270.0,NaN,NaN
4,registration,None,2018-01-01 11:41:27,NaN,47762,147271.0,NaN,NaN


<div class="alert alert-block alert-success">
<b>Вывод:</b> Произвели загрузку данных с БД платформы, применив в запросе интервал для регистрации пользователей с 1 января 2018 до 31 декабря 2018 года. Проверили данные на пропуски, ошибки типизации и наличие дублей - никаких дополнительных преобразований не потребовалось, они готовы к дальнейшей работе.
</div>

**<mark>ЭТАП 2</mark>**  Проверяем гипотезу 1:   

**Отличается ли время прохождения различных этапов для пользователей, которые прошли обучение, от пользователей, не начинавших обучение? Насколько обучение сокращает время прохождения этапов?**

Согласно поставленной задаче, необходимо выделить группы:  
- пользователи, которые прошли обучение (у которых есть событие tutorial_finish)  
- пользователи, которые не начинали обучение (у которых отсутствует событие tutorial_start) 
- пользователи, которые начали, но не окончили обучение (у которых отсутствует событие tutorial_finish) - этого не требовалось, можно считать это нашим бонусом для заказчика  🙂

In [25]:
#Создадим множество пользователей, которые завершили обучение:
set_finish_tutorial_users=set(events_df[events_df['event_type']=='tutorial_finish'].user_id)
len(set_finish_tutorial_users)

12531

In [26]:
#Создадим множество пользователей, которые не начинали обучение:
set_registered_users=set(events_df.user_id)
set_start_tutorial_users=set(events_df[events_df['event_type']=='tutorial_start'].user_id)
set_not_started_tutorial_users=set_registered_users.difference(set_start_tutorial_users)
len(set_not_started_tutorial_users)

9909

In [27]:
#Проверим правильность расчетов:
len(set_registered_users)==len(set_start_tutorial_users)+len(set_not_started_tutorial_users)

True

In [28]:
#Те кто начинал, но не закончил обучение:
set_started_not_finished_users=set_start_tutorial_users.difference(set_finish_tutorial_users)
len(set_started_not_finished_users)

2020

Далее нам необходимо выделить этапы по которым мы будем сравнивать наши группы, поскольку этап прохождения обучения актуален только для двух групп, то мы его не будем анализировать (но он попадет в этап полного пути для групп). По итогу выделяем следующие этапы:  
- start registration -> start level choice (для группы не начавшей обучение), start registration -> tutorial start (для окончивших обучение)
- start level choice -> start training choice  
- start training choice -> start purchase  
- start registration -> start purchase (посмотреть весь путь, не отдаляет ли наше обучение от момента покупки)

Так в первом этапе для группы не начавшей обучение - это время от начала регистрации до выбора уровня сложности, для второй и третьей группы - это время с начала регистрации до начала обучения. Но по сути это время затраченное на регистрацию в приложении до следующего события, поэтому сравнение релевантное.

In [29]:
#Создадим список интересующих нас времен по этапам:
time_stages=['registration_duration', 'level_choice_duration', 'training_choice_duration', 'until_purchase_duration']

In [30]:
#Создадаим датафрейм с интересующими нас данными о времени:
# Мы выбираем минимальное время - то есть самое первое для данного события, если событие могло произойти несколько раз
users_time_df=total_events_df.pivot_table(values=['start_time'], \
                                          index=['user_id'], columns=['event_type'], aggfunc='min').copy()
print(len(users_time_df))
users_time_df.reset_index(inplace=True)
users_time_df.columns=['user_id', 'level_choice', 'purchase', 'registration', 'training_choice', \
                       'tutorial_finish', 'tutorial_start']
users_time_df.head()

24460


,user_id,level_choice,purchase,registration,training_choice,tutorial_finish,tutorial_start
0,47758,NaT,NaT,2018-01-01 04:51:58,NaT,NaT,2018-01-01 13:55:11
1,47759,2018-01-01 20:05:14,NaT,2018-01-01 08:32:05,2018-01-01 20:06:57,2018-01-01 18:06:20,2018-01-01 17:59:28
2,47760,2018-01-01 16:54:36,NaT,2018-01-01 09:30:10,NaT,2018-01-01 14:51:15,2018-01-01 14:45:21
3,47761,2018-01-01 15:05:04,NaT,2018-01-01 09:39:27,2018-01-01 15:12:25,2018-01-01 13:59:44,2018-01-01 13:56:40
4,47762,NaT,NaT,2018-01-01 11:41:27,NaT,NaT,NaT


Для удобства будем давать названия переменных для наших групп с индексами:  
- 1 окончивших обучение  
- 2 не начавших обучение  
- 3 начавших, но не окончивших обучение

In [31]:
#Поскольку нас интересуют разные группы и мы будем датафреймы преобразовывать - разделим его на группы:
users_time_df1=users_time_df[users_time_df['user_id'].isin(set_finish_tutorial_users)].copy()
users_time_df1.reset_index(drop=True, inplace=True)
users_time_df2=users_time_df[users_time_df['user_id'].isin(set_not_started_tutorial_users)].drop(['tutorial_finish', 'tutorial_start'], axis=1).copy()
users_time_df2.reset_index(drop=True, inplace=True)
users_time_df3=users_time_df[users_time_df['user_id'].isin(set_started_not_finished_users)].drop(['tutorial_finish'], axis=1).copy()
users_time_df3.reset_index(drop=True, inplace=True)

In [32]:
# Добавляем интересующие нас колонки с продолжительностью для каждого этапа:
users_time_df1['registration_duration']=(users_time_df1['tutorial_start']-users_time_df1['registration'])
users_time_df1['level_choice_duration']=(users_time_df1['training_choice']-users_time_df1['level_choice'])
users_time_df1['training_choice_duration']=(users_time_df1['purchase']-users_time_df1['training_choice'])
users_time_df1['until_purchase_duration']=(users_time_df1['purchase']-users_time_df1['registration'])

users_time_df2['registration_duration']=(users_time_df2['level_choice']-users_time_df2['registration'])
users_time_df2['level_choice_duration']=(users_time_df2['training_choice']-users_time_df2['level_choice'])
users_time_df2['training_choice_duration']=(users_time_df2['purchase']-users_time_df2['training_choice'])
users_time_df2['until_purchase_duration']=(users_time_df2['purchase']-users_time_df2['registration'])

users_time_df3['registration_duration']=(users_time_df3['tutorial_start']-users_time_df3['registration'])
users_time_df3['level_choice_duration']=(users_time_df3['training_choice']-users_time_df3['level_choice'])
users_time_df3['training_choice_duration']=(users_time_df3['purchase']-users_time_df3['training_choice'])
users_time_df3['until_purchase_duration']=(users_time_df3['purchase']-users_time_df3['registration'])

In [33]:
#Анализ времени прохождения рассматриваемых этапов пользователями прошедших обучение:
users_time_df1.describe(include='timedelta')

,registration_duration,level_choice_duration,training_choice_duration,until_purchase_duration
count,12531,6211,1781,1781
mean,0 days 04:33:30.511291,0 days 00:05:09.668813,3 days 16:59:07.005614,4 days 00:00:26.568781
std,0 days 04:14:38.601502,0 days 00:02:43.241217,2 days 03:37:52.042729,2 days 03:48:24.347282
min,0 days 00:00:38,0 days 00:00:15,0 days 01:31:41,0 days 04:41:08
25%,0 days 01:21:10,0 days 00:02:56,1 days 22:51:29,2 days 05:47:24
50%,0 days 03:16:13,0 days 00:04:47,3 days 12:22:50,3 days 19:12:02
75%,0 days 06:33:24.500000,0 days 00:07:01,5 days 06:56:21,5 days 14:09:55
max,1 days 08:38:10,0 days 00:15:35,10 days 07:56:10,10 days 14:52:09


In [34]:
#Анализ времени прохождения рассматриваемых этапов пользователями даже не начинавших обучение:
users_time_df2.describe(include='timedelta')

,registration_duration,level_choice_duration,training_choice_duration,until_purchase_duration
count,107,77,18,18
mean,0 days 05:10:36.233644,0 days 00:05:31.077922,3 days 23:07:16.111111,4 days 03:33:59.277777
std,0 days 03:25:55.053229,0 days 00:02:46.163027,1 days 22:59:05.990710,1 days 22:09:45.897103
min,0 days 00:05:08,0 days 00:00:11,1 days 00:16:58,1 days 09:14:41
25%,0 days 02:28:16,0 days 00:03:21,2 days 08:16:01,2 days 15:36:56
50%,0 days 04:58:09,0 days 00:05:13,3 days 17:44:32.500000,3 days 21:27:34.500000
75%,0 days 06:56:55,0 days 00:07:28,5 days 07:25:03.750000,5 days 13:10:10.750000
max,0 days 15:06:24,0 days 00:11:49,8 days 08:24:25,8 days 10:36:08


In [35]:
#Анализ времени прохождения рассматриваемых этапов пользователями начавшими, но не окончившими обучение:
users_time_df3.describe(include='timedelta')

,registration_duration,level_choice_duration,training_choice_duration,until_purchase_duration
count,2020,632,175,175
mean,0 days 05:12:39.522772,0 days 00:05:09.458860,3 days 18:44:39.005714,4 days 02:43:05.514285
std,0 days 04:24:28.643158,0 days 00:02:48.079149,2 days 04:49:25.235050,2 days 05:15:54.424986
min,0 days 00:00:49,0 days 00:00:24,0 days 03:37:25,0 days 12:14:03
25%,0 days 01:44:18.250000,0 days 00:02:50.750000,1 days 19:35:41,2 days 03:27:21.500000
50%,0 days 04:05:32.500000,0 days 00:04:46.500000,3 days 13:28:08,3 days 23:07:47
75%,0 days 07:44:48.750000,0 days 00:07:04.250000,5 days 11:48:17.500000,5 days 18:45:05.500000
max,1 days 06:56:46,0 days 00:13:17,8 days 14:19:52,8 days 23:44:36


In [36]:
#Преобразуем интересующие нас колонки с продолжительностью для каждого этапа (необходимо для построения графиков):
users_time_df1['registration_duration']=users_time_df1['registration_duration']/pd.Timedelta('1 hour')
users_time_df1['level_choice_duration']=users_time_df1['level_choice_duration']/pd.Timedelta('1 minute')
users_time_df1['training_choice_duration']=users_time_df1['training_choice_duration']/pd.Timedelta('1 day')
users_time_df1['until_purchase_duration']=users_time_df1['until_purchase_duration']/pd.Timedelta('1 day')

users_time_df2['registration_duration']=users_time_df2['registration_duration']/pd.Timedelta('1 hour')
users_time_df2['level_choice_duration']=users_time_df2['level_choice_duration']/pd.Timedelta('1 minute')
users_time_df2['training_choice_duration']=users_time_df2['training_choice_duration']/pd.Timedelta('1 day')
users_time_df2['until_purchase_duration']=users_time_df2['until_purchase_duration']/pd.Timedelta('1 day')

users_time_df3['registration_duration']=users_time_df3['registration_duration']/pd.Timedelta('1 hour')
users_time_df3['level_choice_duration']=users_time_df3['level_choice_duration']/pd.Timedelta('1 minute')
users_time_df3['training_choice_duration']=users_time_df3['training_choice_duration']/pd.Timedelta('1 day')
users_time_df3['until_purchase_duration']=users_time_df3['until_purchase_duration']/pd.Timedelta('1 day')

Строим box plots каждого этапа для наших групп. Хотелось бы обратить внимание, что для корректного отображения данного вида графиков мы время перевели в минуты, часы, дни (в зависимости от длительности этапа) и они будут отображаться со знаками после запятой как обычные вещественные числа. Это может немного сбивать с толку (как 6,7 часа - непривычно), но общий смысл сохранится. На мой взгляд, именно данный вид графика позволяет более углубленно посмотреть на данные по продолжительности этапов.

In [37]:
x_data=['Прошли обучение', 'Не начинали обучение', 'Начали, но не завершили обучение']
y_data=[users_time_df1['registration_duration'], users_time_df2['registration_duration'], \
        users_time_df3['registration_duration']]

colors = ['rgba(93, 164, 214, 0.5)', 'rgba(255, 144, 14, 0.5)','rgba(127, 96, 0, 0.5)']
fig = go.Figure()

for xd, yd, cls in zip(x_data, y_data, colors):
        fig.add_trace(go.Box(
            y=yd,
            name=xd,
            boxpoints='all',
            jitter=0.2,
            whiskerwidth=0.2,
            fillcolor=cls,
            marker_size=2,
            line_width=1))
fig.update_layout(
    title='Продолжительность регистрации для рассматриваемых групп',
    yaxis=dict(
        title='Часы',
        autorange=True,
        showgrid=True,
        zeroline=True,
        dtick=5,
        gridcolor='rgb(255, 255, 255)',
        gridwidth=1,
        zerolinecolor='rgb(255, 255, 255)',
        zerolinewidth=2,),
    margin=dict(
        l=40,
        r=30,
        b=80,
        t=100,),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    showlegend=False)
fig.show()

<div class="alert alert-block alert-success">
<b>Вывод:</b> Медианное время регистрации из всех групп минимально для группы прошедшей обучение: <b>3,27 часа</b> и максимально для группы не начинавшей обучение - почти <b>5 часов</b>, для группы начавшей, но не завершившей обучение оно составляет <b>4,1 час</b>. В среднем можно сказать, что половина всех пользователей в группе прошедшей обучение проходят его от 1,4 до 6,6 часов, в группе не начинавшей обучение - от 2,5 до 6,9 часов. Также в данном графике обращает внимание большое количество outliers - тех пользователей, которые аномально долго проходили этап регистрации, для групп прошедшей обучение и не завершившей обучение.
</div>

In [38]:
y_data=[users_time_df1['level_choice_duration'], users_time_df2['level_choice_duration'], \
        users_time_df3['level_choice_duration']]

colors = ['rgba(93, 164, 214, 0.5)', 'rgba(255, 144, 14, 0.5)','rgba(127, 96, 0, 0.5)']
fig = go.Figure()

for xd, yd, cls in zip(x_data, y_data, colors):
        fig.add_trace(go.Box(
            y=yd,
            name=xd,
            boxpoints='all',
            jitter=0.2,
            whiskerwidth=0.2,
            fillcolor=cls,
            marker_size=2,
            line_width=1))
fig.update_layout(
    title='Продолжительность выбора уровня сложности тренировок для рассматриваемых групп',
    yaxis=dict(
        title='Минуты',
        autorange=True,
        showgrid=True,
        zeroline=True,
        dtick=1,
        gridcolor='rgb(255, 255, 255)',
        gridwidth=1,
        zerolinecolor='rgb(255, 255, 255)',
        zerolinewidth=2,),
    margin=dict(
        l=40,
        r=30,
        b=80,
        t=100,),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    showlegend=False)
fig.show()

<div class="alert alert-block alert-success">
<b>Вывод:</b> Медианное время выбора уровня тренировки для групп прошедшей обучение и не завершившей практически одникаво: <b>4,8 минут</b> и максимально для группы не начинавшей обучение -  <b>5,2 минуты</b>. В среднем можно сказать, что половина всех пользователей в группе прошедшей обучение проходят его от 2,9 до 7 минут, в группе не начинавшей обучение - от 3,3 до 7,5 минут.
</div>

In [39]:
y_data=[users_time_df1['training_choice_duration'], users_time_df2['training_choice_duration'], \
        users_time_df3['training_choice_duration']]
colors = ['rgba(93, 164, 214, 0.5)', 'rgba(255, 144, 14, 0.5)','rgba(127, 96, 0, 0.5)']
fig = go.Figure()

for xd, yd, cls in zip(x_data, y_data, colors):
        fig.add_trace(go.Box(
            y=yd,
            name=xd,
            boxpoints='all',
            jitter=0.2,
            whiskerwidth=0.2,
            fillcolor=cls,
            marker_size=2,
            line_width=1))
fig.update_layout(
    title='Продолжительность тренировок для рассматриваемых групп',
    yaxis=dict(
        title='Дни',
        autorange=True,
        showgrid=True,
        zeroline=True,
        dtick=1,
        gridcolor='rgb(255, 255, 255)',
        gridwidth=1,
        zerolinecolor='rgb(255, 255, 255)',
        zerolinewidth=2,),
    margin=dict(
        l=40,
        r=30,
        b=80,
        t=100,),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    showlegend=False)
fig.show()

<div class="alert alert-block alert-success">
<b>Вывод:</b> Медианное время тренировок из всех групп минимально для группы прошедшей обучение: <b>3,52 дня</b> и максимально для группы не начинавшей обучение -  <b>3,7 дня</b>, для группы не завершившей обучение оно составляет <b>3,56 дня</b>. В среднем можно сказать, что половина всех пользователей в группе прошедшей обучение проходят его от 2 до 5,3 дня, в группе не начинавшей обучение - от 2,2 до 5,6 дня.
</div>

In [40]:
y_data=[users_time_df1['until_purchase_duration'], users_time_df2['until_purchase_duration'], \
        users_time_df3['until_purchase_duration']]
colors = ['rgba(93, 164, 214, 0.5)', 'rgba(255, 144, 14, 0.5)','rgba(127, 96, 0, 0.5)']
fig = go.Figure()

for xd, yd, cls in zip(x_data, y_data, colors):
        fig.add_trace(go.Box(
            y=yd,
            name=xd,
            boxpoints='all',
            jitter=0.2,
            whiskerwidth=0.2,
            fillcolor=cls,
            marker_size=2,
            line_width=1))
fig.update_layout(
    title='Продолжительность всего пути с момента регистрации и до покупки тренировок для  рассматриваемых групп',
    yaxis=dict(
        title='Дни',
        autorange=True,
        showgrid=True,
        zeroline=True,
        dtick=1,
        gridcolor='rgb(255, 255, 255)',
        gridwidth=1,
        zerolinecolor='rgb(255, 255, 255)',
        zerolinewidth=2,),
    margin=dict(
        l=40,
        r=30,
        b=80,
        t=100,),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    showlegend=False)
fig.show()

<div class="alert alert-block alert-success">
<b>Вывод:</b> Медианное время всего пути с момента регистрации и до оплаты из всех групп минимально для группы прошедшей обучение: <b>3,8 дня</b> и максимально для группы не завершившей обучение - почти <b>4 дня</b>, для группы не начинавшей обучение оно составляет <b>3,9 дня</b>. В среднем можно сказать, что половина всех пользователей в группе прошедшей обучение проходят весь путь до покупки от 2,2 до 5,6 дня, в группе не начинавшей обучение - от 2,6 до 5,8 дня, в группе начавшей, но не завершившей обучение - от 2,1 до 5,8 дня. 
</div>

<div class="alert alert-block alert-warning">
<b>Вывод по гипотезе 1:</b>  Наша гипотеза подтверждается: медианное значение для всего пути от регистрации и до оплаты, а также на всех его рассматриваемых этапах пользователи, которые не проходили обучение, тратят больше времени на его прохождение, чем пользователи прошедшие обучение на <b>2,3%</b> и пользователи, которые начали, но не завершили обучение проходят его на <b>4,2%</b> дольше прошедших обучение. 
</div>

**<mark>ЭТАП 3</mark>**  Проверяем гипотезу 2:   

**Существует ли зависимость между вероятностью оплаты тренировки и количеством обучений, которые начинал или завершал пользователь. Правда ли, что успешное обучение влияет на оплату само по себе, без разницы, какое оно было по порядку?**

Попробуем сделать воронку прохождения этапов пользователями, согласно нашим группам.

In [41]:
#Делаем список всех существующих этапов:
stages=total_events_df['event_type'].unique()

Помним, что для удобства дали названия переменных для наших групп с индексами:  
- 1 окончивших обучение  
- 2 не начавших обучение  
- 3 начавших, но не окончивших обучение

In [42]:
#Построим доли наших групп среди всех уникальных пользователей:
data=go.Pie(labels = ['Прошли обучение', 'Начали, но не завершили обучение', 'Не начинали обучение'], 
            values = [len(set_finish_tutorial_users), len(set_started_not_finished_users), len(set_not_started_tutorial_users)])
layout=go.Layout(title = 'Доля наших групп среди уникальных зарегистрировавшихся пользователей')

fig=go.Figure(data, layout)
fig.update_traces(textposition='inside', textinfo='percent+value')
fig.update_layout(uniformtext_minsize=9, uniformtext_mode='hide')

fig.show()

In [43]:
#Определим количество пользователей на каждом этапе для наших групп (при этом для каждой группы будут свои этапы):
set1_registation_users=set([i for i in total_events_df[total_events_df['event_type']=='registration'].user_id \
                            if i in set_finish_tutorial_users])
set1_tutorial_start_users=set([i for i in total_events_df[total_events_df['event_type']=='tutorial_start'].user_id \
                               if i in set_finish_tutorial_users])
set1_level_choice_users=set([i for i in total_events_df[total_events_df['event_type']=='level_choice'].user_id \
                             if i in set_finish_tutorial_users])
set1_training_choice_users=set([i for i in total_events_df[total_events_df['event_type']=='training_choice'].user_id \
                                if i in set_finish_tutorial_users])
set1_purchase_users=set([i for i in total_events_df[total_events_df['event_type']=='purchase'].user_id \
                         if i in set_finish_tutorial_users])

values1=[len(set1_registation_users), len(set1_tutorial_start_users), len(set_finish_tutorial_users), \
         len(set1_level_choice_users), len(set1_training_choice_users), len(set1_purchase_users)]

In [44]:
set2_registation_users=set([i for i in total_events_df[total_events_df['event_type']=='registration'].user_id \
                            if i in set_not_started_tutorial_users])
set2_level_choice_users=set([i for i in total_events_df[total_events_df['event_type']=='level_choice'].user_id \
                             if i in set_not_started_tutorial_users])
set2_training_choice_users=set([i for i in total_events_df[total_events_df['event_type']=='training_choice'].user_id \
                                if i in set_not_started_tutorial_users])
set2_purchase_users=set([i for i in total_events_df[total_events_df['event_type']=='purchase'].user_id \
                         if i in set_not_started_tutorial_users])

values2=[len(set2_registation_users), 0, 0, len(set2_level_choice_users), len(set2_training_choice_users), \
         len(set2_purchase_users)]

In [45]:
set3_registation_users=set([i for i in total_events_df[total_events_df['event_type']=='registration'].user_id \
                            if i in set_started_not_finished_users])
set3_level_choice_users=set([i for i in total_events_df[total_events_df['event_type']=='level_choice'].user_id \
                             if i in set_started_not_finished_users])
set3_training_choice_users=set([i for i in total_events_df[total_events_df['event_type']=='training_choice'].user_id \
                                if i in set_started_not_finished_users])
set3_purchase_users=set([i for i in total_events_df[total_events_df['event_type']=='purchase'].user_id \
                         if i in set_started_not_finished_users])

values3=[len(set3_registation_users), len(set_started_not_finished_users), 0, len(set3_level_choice_users), \
         len(set3_training_choice_users), len(set3_purchase_users)]

In [46]:
fig = go.Figure()

trace1 = fig.add_trace(go.Funnel(
    name = 'Окончили обучение',
    y = stages,
    x = values1,
    textinfo = "value+percent initial"))

trace2 = fig.add_trace(go.Funnel(
    name = 'Не начали обучение',
    orientation = "h",
    y = stages,
    x = values2,
    textposition = "inside",
    textinfo = "value+percent previous"))

trace3 = fig.add_trace(go.Funnel(
    name = 'Начали обучение, но не окончили',
    orientation = "h",
    y = stages,
    x = values3,
    textposition = "outside",
    textinfo = "value+percent total"))

fig.update_layout(title_text='Воронка пользователей по этапам согласно выделенным группам')
fig.show()

<div class="alert alert-block alert-success">
<b>Вывод:</b> Визуализация прохождения этапов наших рассматриваемых групп демонстрирует, что обучение <b>очень сильно влияет</b> на процент пользователей выбирающих уровень сложности тренировок, сами тренировки и выбор платных тренировок. Можно также предположить, что не обучавшиеся пользователи  скачивали наше приложение скорее из любопытства и хотели посмотреть на приложение, либо те кто так и не смог заставить себя заставить заниматься спортом (что говорить - это очень тяжело!). Те кто начал тренировки, но не окончил также в меньшем процентном соотношении проходят последующие этапы, чем полностью прошедшие обучение. Также показательны проценты оплативших по группам по сравнению с первоначальными зарегистрированными: <b>0,2%</b> для группы не начавшей обучение, <b>8,7%</b> для группы начавшей, но не завершившей обучение и <b>14,2%</b> для группы завершившей обучение.
</div>

In [47]:
#Создадим датафрейм с необходимыми нам параметрами:
tut_attemps_df=total_events_df.groupby('user_id')['tutorial_id'].nunique().reset_index()
tut_attemps_df.columns=['user_id', 'tutorial_attemps']

#Создадим колонку finish_tutorial со значением 1 если окончил обучеие, 0 - если не окончил:
tut_attemps_df['finish_tutorial']=tut_attemps_df.apply(lambda x: 1 if x['user_id'] in set_finish_tutorial_users \
                                                         else 0, axis=1)

#Создадим колонку purchase со значением 1 если купил платные тренировки, 0 - если покупки не было:
set_purchase_users=set(purchase_df['user_id'])
tut_attemps_df['purchase']=tut_attemps_df.apply(lambda x: 1 if x['user_id'] in set_purchase_users \
                                                         else 0, axis=1)

In [48]:
# Посмотрим что у нас получилось:
tut_attemps_df.head()

,user_id,tutorial_attemps,finish_tutorial,purchase
0,47758,2,0,0
1,47759,2,1,0
2,47760,2,1,0
3,47761,3,1,0
4,47762,1,0,0


In [49]:
#Поскольку по условию нас интересует только успешное обучение (то есть завершенное), преобразуем наш датафрейм:
tut_attemps_df=tut_attemps_df[tut_attemps_df.finish_tutorial==1].drop(['finish_tutorial'], axis=1)
tut_attemps_df.head()

,user_id,tutorial_attemps,purchase
1,47759,2,0
2,47760,2,0
3,47761,3,0
8,47766,2,0
10,47768,3,0


In [50]:
tut_attemps_df=tut_attemps_df.merge(purchase_df, how='left', on='user_id')\
.drop(['start_time', 'purchase_id', 'event_type'], axis=1)

In [51]:
# Преобразуем датафрейм сразу меняя названия колонок:
grouped_df=tut_attemps_df.groupby('tutorial_attemps').agg(users=pd.NamedAgg(column='user_id', aggfunc='count'),\
                                                           purchase_users=pd.NamedAgg(column='purchase', aggfunc='sum'), \
                                                           amount=pd.NamedAgg(column='amount', aggfunc='sum'))

In [52]:
grouped_df

,users,purchase_users,amount
tutorial_attemps,,,
2,9101,1295,141100.0
3,1955,265,28300.0
4,528,82,9100.0
5,227,35,3725.0
6,163,25,2375.0
7,138,15,1725.0
8,143,23,2125.0
9,123,19,2050.0
10,153,22,1975.0


In [53]:
#Подсчитаем конверсию и средний чек:
grouped_df['conversion']=grouped_df['purchase_users']/grouped_df['users']
grouped_df['avg_bill']=grouped_df['amount']/grouped_df['purchase_users']
grouped_df

,users,purchase_users,amount,conversion,avg_bill
tutorial_attemps,,,,,
2,9101,1295,141100.0,0.142292,108.957529
3,1955,265,28300.0,0.135550,106.792453
4,528,82,9100.0,0.155303,110.975610
5,227,35,3725.0,0.154185,106.428571
6,163,25,2375.0,0.153374,95.000000
7,138,15,1725.0,0.108696,115.000000
8,143,23,2125.0,0.160839,92.391304
9,123,19,2050.0,0.154472,107.894737
10,153,22,1975.0,0.143791,89.772727


In [54]:
# Найдем медианной значение конверсии:
print('Медианное значение конверсии для пользователей, закончивших обучение: {} %'.\
      format(round(grouped_df['conversion'].median()*100, 1)))

Медианное значение конверсии для пользователей, закончивших обучение: 15.3 %


In [55]:
#Построим стобчатые диаграммы с конверсиями:
fig = go.Figure()

data=[go.Bar(
    x=grouped_df.index, 
    y=list(round(grouped_df.conversion*100, 1)),  
    text=round(grouped_df.conversion*100, 1),
    texttemplate='%{text:.1f}%',
    textposition='auto', 
    width=0.5,
    marker_color='lightsalmon')]

layout=go.Layout(title='Конверсии пользователей в зависимости от количества повторных обучений', 
                yaxis_title='Конверсия, %',
                xaxis_title='Количество повторных обучений')

fig=go.Figure(data, layout)

fig.show()

In [56]:
#Построим столбчатые диаграммы со средним чеком:
fig = go.Figure()

data=[go.Bar(
    x=grouped_df.index, 
    y=list(round(grouped_df.avg_bill, 1)),
    name='Средний чек',
    text=round(grouped_df.avg_bill, 1),
    textposition='auto', 
    width=0.5,
    marker_color='teal')]

layout=go.Layout(title='Средний чек в зависимости от количества повторных обучений', 
                yaxis_title='Средний чек, рубли',
                xaxis_title='Количество повторных обучений')

fig=go.Figure(data, layout)

fig.show()

<div class="alert alert-block alert-success">
<b>Вывод:</b> Медианная конверсия составляет 15,3%. Как видно из столбчатой диаграммы  по количествам повторных обучений - только на количестве повторных обучений 6 раз конверсия сильно отличается в меньшую сторону, в остальных группах значения достаточно близки к медианному. Максимальная конверсия для группы пользователей, совершивших обучение 7 раз.Что касается среднего чека, то имеется слабая тенденция его уменьшения при увеличении повторения обучения. Интересно, что его максимальное начение в 115 рублей приходится на 6 повторений, где минимальная конверсия. 
</div> 

In [57]:
#Построим круговую диаграмму с долями пользователей и их выручкой для тех, кто прошел обучение:
labels = ['Группа '+str(i) for i in grouped_df.index]

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=labels, values=grouped_df.purchase_users, name="Доля пользователей"),
              1, 1)
fig.add_trace(go.Pie(labels=labels, values=grouped_df.amount, name="Доля выручки"),
              1, 2)

fig.update_traces(hole=.4, textposition='inside',
              textinfo='percent+value', hoverinfo="label+percent+value")

fig.update_layout(
    title_text="Доля пользователей и их выручка среди тех, кто прошел обучение и купил платные тренировки",
    annotations=[dict(text='Пользователи', x=0.145, y=0.5, font_size=18, showarrow=False),
                 dict(text='Выручка', x=0.83, y=0.5, font_size=18, showarrow=False)])
fig.show()

<div class="alert alert-block alert-success">
<b>Вывод:</b> Видим, что 72,7% пользователей, которые принесли нам 73,3% выручки - проходили обучение один раз.  
</div> 

<div class="alert alert-block alert-warning">
<b>Вывод по гипотезе 2:</b>  Среди зарегистрировавшихся уникальных пользователей 51,2 % тех, кто завершил обучение, 40,5% - кто даже не начинал обучение и 8,26% - кто начинал, но по каким-то причинам его не окончил. Наиболее информативна визуализация прохождения этапов пользователя в виде воронки продаж. Так вероятность оплаты среди группы окончившей обучение составляет <b>14,2%</b>, для группы начавшей, но не завершившей обучение <b>8,7%</b> и для группы не начавшей обучение - всего <b>0,2% (!)</b>. Также мы выдвинули предположение, что в группу не обучавшихся пользователей попали те, кто скачивал наше приложение скорее из любопытства и хотели посмотреть на приложение, либо те кто так и не смог заставить себя заставить заниматься спортом. Но это также может свидетельствовать, что для данных пользователей с начала регистрации возникали трудности с использованием приложения. Анализ конверсии по группам согласно повторным обучениям показал, что успешное обучение влияет на оплату само по себе, независимо от того, какое оно было по порядку. Анализ среднего чека все же имеет минимальную тенденцию к уменьшению в зависимости от количества повторных обучений. Данные выводы позволяют нам подтвердить нашу вторую гипотезу.
</div>

**<mark>ЭТАП 4</mark>**  Проверяем гипотезу 3:   

**Как часто пользователи начинают обучение после того, как они выбрали уровень сложности тренировок?**

In [58]:
#Попробуем воспользоваться нашим готовым датафреймом с первыми событиями времени по этапам для всех пользователей:
users_time_df[(users_time_df.tutorial_start-users_time_df.level_choice)>pd.Timedelta(0)]

,user_id,level_choice,purchase,registration,training_choice,tutorial_finish,tutorial_start


Отсутствие значений в полученном результате свидетельствует, что все пользователи, выбравшие уровень сложности тренировок, первоначально проходили обучение. Попробуем проанализировать возможные варианты, когда существуют события возврата к обучению после выбора уровня сложности тренировки с использованием первоначального датафрейма events_df.  

Создадим два датафрейма tutorial_start_df и level_choice_df только с интересующими нас колонками, объединим их по пользователям и найдем разницу во времени между началом обучения и выбором сложности уровня тренировок. Оставим только те записи, где она положительная (где обучение после выбора уровня сложности тренировок) - получили датафрейм с интересующими нас пользователями.

In [59]:
tutorial_start_df=events_df[events_df['event_type']=='tutorial_start'][['start_time', 'user_id']]\
.rename(columns={'start_time':'tutorial_start_time'})
level_choice_df=events_df[events_df['event_type']=='level_choice'][['start_time', 'user_id']]\
.rename(columns={'start_time':'level_choice_time'})
focus_group_df=pd.merge(tutorial_start_df, level_choice_df, on='user_id')
focus_group_df['timedelta']=focus_group_df['tutorial_start_time']-focus_group_df['level_choice_time']
focus_group_df=focus_group_df[focus_group_df.timedelta>pd.Timedelta(0)].reset_index(drop=True)

In [60]:
#Посмотрим на итоговый датафрейм:
focus_group_df.head()

,tutorial_start_time,user_id,level_choice_time,timedelta
0,2018-01-01 20:41:53,47761,2018-01-01 15:05:04,05:36:49
1,2018-01-01 20:04:42,47768,2018-01-01 18:44:32,01:20:10
2,2018-01-03 04:29:18,47793,2018-01-02 21:02:30,07:26:48
3,2018-01-03 19:30:20,47824,2018-01-03 17:46:48,01:43:32
4,2018-01-04 09:08:56,47851,2018-01-04 04:03:32,05:05:24


In [61]:
#Посмотрим на количество возвратов к обучению:
repeat_tutorial_df=focus_group_df.user_id.value_counts().reset_index().rename(columns={'user_id':'repeat_tutor_count'})
repeat_tutorial_df['repeat_tutor_count'].value_counts()

1    1048
2     261
3     146
4     116
5      89
6      60
7      37
8      12
Name: repeat_tutor_count, dtype: int64

In [62]:
#Всего случаев выбора начала обучения после выбора уровня тренировок:
len(focus_group_df)

3632

In [63]:
#Медианное время, понадобившиеся для осознания необходимости заново пройти обучение:
focus_group_df.timedelta.median()

Timedelta('0 days 03:43:18')

In [64]:
#Количество и процент от общего числа ошибившихся пользователей
print('Количество пользователей выбравших обучение после выбора уровня тренировок: {}'.format(focus_group_df.user_id.nunique()))
print('Процент данных пользователей от общего числа зарегистрированных уникальных пользователей: {:.2%}'\
      .format(focus_group_df.user_id.nunique()/events_df.user_id.nunique()))
print('Процент данных пользователей от тех, кто проходил обучение: {:.2%}'\
      .format(focus_group_df.user_id.nunique()/len(set_start_tutorial_users)))

Количество пользователей выбравших обучение после выбора уровня тренировок: 1769
Процент данных пользователей от общего числа зарегистрированных уникальных пользователей: 7.23%
Процент данных пользователей от тех, кто проходил обучение: 12.16%


<div class="alert alert-block alert-warning">
<b>Вывод по гипотезе 3:</b> Согласно медианному времени, данным пользователям требуется <b>3 часа 43 минуты</b>, чтобы понять что приложение им непонятно и все же необходимо заново пройти обучение. Так существуют <b>3632</b> случая перехода к обучению после выбора уровня сложности тренировок. Это <b>1769</b> уникальных пользователя, что составляют <b>7,23%</b> от всех зарегистрировавшихся пользователей. В предыдущей гипотезе было доказано, что обучение сильно влияет на прохождение последующих этапов и, соответственно на последующиую конверсию. А доля таких "непонятливых" пользователей среди начавших обучение - <b>12,16%</b>, что в целом, не так уж и мало, чтобы пренебрегать данными цифрами. Также тот факт, что все пользователи первоначально все же проходили обучение, но потом к нему все же возвращались (некоторые аж до 8 раз) подтверждает нашу гипотезу о возникновении трудности в пользовании нашим приложении у ряда пользователей либо возможно процесс обучения стоит усовершенствовать. 
</div>

<font size="6"><code style="background:teal;color:white">Выводы по проекту:</code></font>

<div class="alert alert-block alert-warning"><b>Гипотеза 1 :</b>Отличается ли время прохождения различных этапов для пользователей, которые прошли обучение, от пользователей, не начинавших обучение? Насколько обучение сокращает время прохождения этапов?</div>

В результате анализа были получены следующие результаты:

<table>
<thead>
<tr><th>Группа</th><th>Продолжительность регистрации</th><th>Продолжительность выбора уровня</th><th>Продолжительность выбора тренировки</th><th>Продолжительность всего пути до покупки</th></tr>
</thead>
<tbody>
<tr><td>Прошли обучение</td><td>3 час 16 мин</td><td>4 мин 47 сек</td><td>3 дня 12 час 23 мин</td><td>3 дня 19 час 12 мин</td></tr>
<tr><td>Не обучались</td><td>4 час 58 мин</td><td>5 мин 13 сек</td><td>3 дня 17 час 45 мин</td><td>3 дня 21 час 28 мин</td></tr>
<tr><td>Не окончили</td><td>4 час 05 мин</td><td>4 мин 46 сек</td><td>3 дня 13 час 28 мин</td><td>3 дня 23 час 08 мин</td></tr>
</tbody>
</table>

<blockquote>
<p><b>Вывод:</b> пользователи, которые прошли до конца обучения, по медианным показателям весь путь проходят на <b>2 часа 16 минут (2,3%)</b> быстрее тех, кто не начинал обучение (с учетом даже времени на само обучение), и на <b>3 часа 56 минут (4,2%)</b> быстрее тех, кто начал, но не закончил обучение. Последний факт очень интересен и свидетельствует о том, что важно именно завершить обучение. Также наличие большого числа outliers (выбросов) на этапе регистрации свидетельствует о большом количестве "зависших" пользователей. <br><b>Рекомендации:</b> рассмотреть вариант <b>push-уведомлений</b> для пользователей превысившим медианное время с начала регистрации для стимулирования к использованию приложения. Также рассмотреть вариант стимулирования именно завершения обучения, например, предложив скидки на самые дорогие тренировки по окончанию обучения (заодно это даст возможность пользователям почувствовать разницу между бесплатными и платными тренировками). Возможно это также сможет увеличить конверсию, которая с прошлого года осталась практически неизменной <b>(8,07%)</b>. </p>
</blockquote>

<div class="alert alert-block alert-warning"><b>Гипотеза 2 :</b>Существует ли зависимость между вероятностью оплаты тренировки и количеством обучений, которые начинал или завершал пользователь. Нужно доказать, что успешное обучение влияет на оплату само по себе, без разницы, какое оно было по порядку.</div>

По итогам анализа были получены следующие результаты по прохождению этапов (в процентном отношении дана доля от первоначально зарегистрированных для данной группы):

<table>
<thead>
<tr><th>Этап</th><th>Завершили обучение (чел / %)</th><th>Не обучались (чел / %)</th><th>Не завершили обучение (чел/ %)</th></tr>
</thead>
<tbody>
<tr><td>Регистрация</td><td>12 531 (100%)</td><td>9 909 (100%)</td><td>2 020 (100%)</td></tr>
<tr><td>Начало обучения</td><td>12 531 (100%)</td><td> - </td><td>2 020 (100%)</td></tr>
<tr><td>Конец обучения</td><td>12 531 (100%)</td><td> - </td><td> - </td></tr>
<tr><td>Выбор уровня тренировок</td><td>9 132 (72,9%)</td><td>107 (1,1%)</td><td>959 (47,5%)</td></tr>
<tr><td>Выбор тренировок</td><td>6 211 (49,6%)</td><td>77 (0,8%)</td><td>632 (31,3%)</td></tr>
<tr><td>Покупка</td><td>1 781 (14,2%)</td><td>18 (0,2%)</td><td>175 (8,7%)</td></tr>
</tbody>
</table>

Также получили данные о количествах повторных обучений, пользователях, конверсии и среднему чеку к каждому количеству повторений:

<table>
<thead>
<tr><th>Количество повторений</th><th>Пользователи (чел)</th><th>Покупатели (чел)</th><th>Сумма (руб)</th><th>Конверсия (%)</th><th>Средний чек (руб)</th></tr>
</thead>
<tbody>
<tr><td>1</td><td>9 101</td><td>1 295</td><td>141 100</td><td>14,2</td><td>109</td></tr>
<tr><td>2</td><td>1 955</td><td>265</td><td>28 300</td><td>13,6</td><td>106,8</td></tr>
<tr><td>3</td><td>528</td><td>82</td><td>9 100</td><td>15,5</td><td>111</td></tr>
<tr><td>4</td><td>227</td><td>35</td><td>3 725</td><td>15,4</td><td>106,4</td></tr>
<tr><td>5</td><td>163</td><td>25</td><td>2 375</td><td>15,3</td><td>95</td></tr>
<tr><td>6</td><td>138</td><td>15</td><td>1 725</td><td>10,9</td><td>115</td></tr>
<tr><td>7</td><td>143</td><td>23</td><td>2 125</td><td>16,1</td><td>92,4</td></tr>
<tr><td>8</td><td>123</td><td>19</td><td>2 050</td><td>15,4</td><td>107,9</td></tr>
<tr><td>9</td><td>153</td><td>22</td><td>1 975</td><td>14,4</td><td>89,8</td></tr>
</tbody>
</table>

<blockquote>
<p><b>Вывод:</b> данные по прохождениям этапов нашего приложения для рассматриваемых групп пользователей свидетельствуют о большой зависимости вероятности оплаты тренировки от прохождения обучения. Так, для пользователей прошедших обучения она составляет <b>14,2%</b> от первоначально зарегистрированных, для не закончивших обучение - <b>8,7%</b> и для не начинавших обучение - всего <b>0,2%</b>. С одной стороны, это свидетельствует о важности этапа обучения, с другой - всегда находятся пользователи, которые регистрируются в приложениях из любопытства, либо которые так и не смогли заняться спортом, тогда они и будут попадать в данную группу. Также мы рассмотрели конверсию в зависимости от количества повторных обучений - она достаточна постоянна и независима, что подтверждает об влиянии успешного обучения на оплату само по себе, без разницы, какое оно было по порядку. Средний чек имеет минимальную тенденцию к уменьшению с увеличением количества повторений.<br>
<b>Рекомендации:</b> необходим более детальный анализ группы не начинавшей обучение (это все-таки <b>40,5%</b> наших пользователей, которые могут принести дополнительную выручку). Важно понять причину по которой пользователи с самого начала работы с нашим приложением не переходят на последующие уровни: им не нравится само приложение, они не смогли себя замотивировать на тренировки, этап регистрации вызывает трудности и они его не завершают, они не увидели ценности продукта для себя...</p>
</blockquote>

<div class="alert alert-block alert-warning"><b>Гипотеза 3 :</b>Как часто пользователи начинают обучение после того, как они выбрали уровень сложности тренировок? Это позволит нам понять, насколько процесс работы с приложением понятен для пользователей: если пользователи после выбора уровня сложности обращаются к обучению, значит, работа с приложением непонятна.</div>
 
Провели анализ по количествам возврата к обучению после выбора уровня сложности тренировки:

<table>
<thead>
<tr><th>Количество возвратов к обучению</th><th>Пользователи (чел)</th></tr>
</thead>
<tbody>
<tr><td>1</td><td>1 048</td></tr>
<tr><td>2</td><td>261</td></tr>
<tr><td>3</td><td>146</td></tr>
<tr><td>4</td><td>116</td></tr>
<tr><td>5</td><td>89</td></tr>
<tr><td>6</td><td>60</td></tr>
<tr><td>7</td><td>37</td></tr>
<tr><td>8</td><td>12</td></tr>
<tr><td>Итого</td><td>1 769</td></tr>
</tbody>
</table>

Всего уникальных пользователей: **24 460**  
Всего случаев возврата к обучению: **3632**  
Количество уникальных пользователей, вернувшихся к обучени: **1769**  
Процент пользователей, вернувшихся к обучению от числа зарегистрированных: **7,23%**  
Процент пользователей, вернувшихся к обучению от числа прошедших обучение: **12,16%**  
Медианное время, требующееся для осознания необходимости возврата к обучению: **3 часа 43 минуты**  

<blockquote>
<p><b>Вывод:</b> в первую очередь необходимо отметить, что абсолютно все обучавшиеся пользователи при первом проходе по этапам проходили сначала обучение, а затем выбор уровня. По-каким-то причинам <b>12,16%</b> из них (или <b>7,23%</b> от числа зарегистрированных) возвращались к обучению повторно, некотрые даже до <b>8</b> раз. Это достаточно большой процент пользователей, чтобы пренебречь данным моментом.<br>
<b>Рекомендации:</b> сам процесс обучения достаточно короткий - около <b>5 минут</b>, необходимо его проанализировать - понятно ли как тренироваться по этому приложению за такой короткий промежуток времени. Также возможно проблема в сложности использования самого приложения - недружелюбный интерфейс, не понятно как выбирать упражнения, оплачивать и т.д. Возможно стоит разбить обучение на отрезки относящиеся к каждому этапу и вставить его перед каждым этапом по мере прохождения.</p>
</blockquote>

<div class="alert alert-block alert-success">
<b>Дополнительно:</b> В реальном кейсе, конечно, необходимо ознакомиться с продуктом и пройти самому все этапы пользователя - это даст возможность более подробного анализа и рекомендаций. Также смутила малая сумма выручки от продукта (предполагаю, что это только развивающийся продукт, но все же). Необходимо исследовать дополнительные возможности монетизации:<ol><li>Включить новые продукты в приложение (программы питания, консультации тренеров, музыкальное сопровождение, подключение к фитнесс браслетам, даже включение исскуственного интеллекта, контролирующего правильность положения тела при выполнении упражнений (стартап уже существует и пытается создать  такой продукт на рынке))</li><li>Добавление рекламы (зарабатывать также или на рекламе, или на возможности пользования приложением без рекламы)</li><li>Сделать приложение платным для всех (но при этом добавить ценность для пользователей)</li></ol>
</div> 